In [ ]:
import json
import numpy as np
import skimage
import tifffile
import os
import shutil

def create_mask(image_info, annotations, output_folder):
    # Create an empty mask as a numpy array
    mask_np = np.zeros((image_info['height'], image_info['width']), dtype=np.uint8)

    for ann in annotations:
        if ann['image_id'] == image_info['id']:
            # Extract segmentation polygon
            for seg in ann['segmentation']:
                vertices_y_float = np.array(seg[1::2])
                vertices_x_float = np.array(seg[0::2])

                vertices_y_int = np.round(vertices_y_float).astype(int)
                vertices_x_int = np.round(vertices_x_float).astype(int)

                # Draw the polygon on the mask using integer coordinates
                rr, cc = skimage.draw.polygon(vertices_y_int, vertices_x_int, mask_np.shape)
                mask_np[rr, cc] = 255

    mask_np = np.clip(mask_np, 0, 255)

    # Save the numpy array as a TIFF using tifffile library
    mask_path = os.path.join(output_folder, image_info['file_name'].replace('.jpg', '_mask.jpg'))
    tifffile.imsave(mask_path, mask_np)

    print(f"Saved mask for {image_info['file_name']} to {mask_path}")


def main(json_file, mask_output_folder, image_output_folder, original_image_dir):
    # Load COCO JSON annotations
    with open(json_file, 'r') as f:
        data = json.load(f)

    images = data['images']
    annotations = data['annotations']

    # Ensure the output directories exist
    if not os.path.exists(mask_output_folder):
        os.makedirs(mask_output_folder)
    if not os.path.exists(image_output_folder):
        os.makedirs(image_output_folder)

    for img in images:
        # Create the masks
        create_mask(img, annotations, mask_output_folder)
        
        # Copy original images to the specified folder
        original_image_path = os.path.join(original_image_dir, img['file_name'])
    
        new_image_path = os.path.join(image_output_folder, os.path.basename(original_image_path))
        shutil.copy2(original_image_path, new_image_path)
        print(f"Copied original image to {new_image_path}")


if __name__ == '__main__':
    original_image_dir = 'data/roi_train/'  # Where your original images are stored
    json_file = 'data/roi_train/_annotations.coco.json'
    mask_output_folder = 'train/masks'  # Modify this as needed. Using val2 so my data is not overwritten
    image_output_folder = 'train/images'  # 
    main(json_file, mask_output_folder, image_output_folder, original_image_dir)